# 2. Dataset Description

In [1]:
# Import necessary libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from matplotlib.pyplot import figure
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
InteractiveShell.ast_node_interactivity = "all"
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('Cost Prediction - Litigation.csv')

In [3]:
TP = 150 #TPPP

In [4]:
def accuracy(TP,df):
    #Creating a dictionary recording all the postions from EMPL_UNO and POSITIONS
    df_pos = dict(list(zip(df.EMPL_UNO.astype(str), df.POSITION)))
    df = df.replace(' ', np.nan)
    df = df.replace(' .', np.nan)
    #import pickle files
    objects = []
    with (open("C://python working//Dissertation Project//pos_dict.pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

    with (open("C://python working//Dissertation Project//action_dict_updated.pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

    with (open("C://python working//Dissertation Project//task_dict_updated.pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

    with (open("C://python working//Dissertation Project//task_phase.pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    # Mapped POSITION values from pickle files
    df.replace({'POSITION': objects[0]}, inplace = True)
    # Mapped ACTION_DESC values from pickle files
    df.replace({'ACTION_DESC': objects[1]}, inplace = True)
    # Mapped ACTION_DESC values from pickle files
    df.replace({'PHASE_DESC': objects[2]}, inplace = True)
    # Mapped ACTION_DESC values from pickle files
    df.replace({'TASK_DESC': objects[3]}, inplace = True)
    # Replace all the NaN values in PHASE_DESC and TASK_DESC by Missing values
    df['PHASE_DESC'].fillna('Missing', inplace=True)
    df['TASK_DESC'].fillna('Missing', inplace=True)
    #Creating main datatable according to unique ID number
    df_main = pd.DataFrame(df['MATTER_UNO'].unique())
    #Rename column
    df_main.rename(columns={0: 'MATTER_UNO'}, inplace = True)
    # Creating TOTAL_COST column
    df_cost = df[['MATTER_UNO','BILLED_AMT']].groupby(['MATTER_UNO']).sum().rename(columns={'BILLED_AMT': 'TOTAL_AMT'})
    # #Total cost of each matter
    # df_cost
    #Merge total cost of each matter to main table
    df_main = df_main.merge(df_cost, on='MATTER_UNO', how = 'left')
    # We have 236 matters are 0 cost in total, which can be dropped.
    df_main = df_main.drop(df_main[df_main.TOTAL_AMT==0].index).reset_index().drop(columns = 'index', axis=1)
    # Create VALUE_RANGE variable and check the number of people in each range
    df_main['VALUE_RANGE'] = pd.DataFrame(pd.qcut(df_main['TOTAL_AMT'], 4,labels = False))
    # Check the total matter numbers in different VALUE_RANGE
    df_val = df_main[['MATTER_UNO','VALUE_RANGE']].groupby('MATTER_UNO').mean().groupby('VALUE_RANGE').size()
    # Creating DAYS_SPENT
    df['DAYS_SPENT'] = pd.to_datetime(df['CLOSE_DATE']) - pd.to_datetime(df['OPEN_DATE'])
    # get the earliest DATE among all the TRAN_DATE to each matter
    df_start = df[['MATTER_UNO','TRAN_DATE']].groupby('MATTER_UNO').min().rename(columns={'TRAN_DATE':'FIRST_TRAN_DATE'})
    df = df.merge(df_start, on='MATTER_UNO', how = 'left')
    #Get the earliest date between FIRST_TRAN_DATE and OPEN_DATE as the START_DATE to eachmatter
    df_start = df[['MATTER_UNO','TRAN_DATE', 'FIRST_TRAN_DATE']].groupby('MATTER_UNO').min()
    # Merge the earliet TRAN_DATE AS START_DATE to the datetable
    df_start=df_start.rename(columns={'TRAN_DATE' : 'START_DATE'})
    df = df.merge(df_start, on='MATTER_UNO', how = 'left')
    # Drop the Dates used to find the earliest start_date
    df.drop(columns = ['FIRST_TRAN_DATE_x', 'FIRST_TRAN_DATE_y'], inplace = True)
    # Creating new DAYS_SPENT, as the days spent should be the one from ealiest START_DATE to final CLOSE_DATE.
    df['DAYS_SPENT'] = pd.to_datetime(df['CLOSE_DATE']) - pd.to_datetime(df['START_DATE'])
    # Convert days to numerical number.
    df['DAYS_SPENT'] = df['DAYS_SPENT'].astype('timedelta64[D]')
    # Firstly, Merge DAYS_SPENT to each matter to main table
    df_days = df[['MATTER_UNO','DAYS_SPENT']].groupby('MATTER_UNO').mean()
    df_main = df_main.merge(df_days, on='MATTER_UNO', how = 'left')
    # Define MAPE (Mean absolute percentage error) to check regression accuracy
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    # Save all matters more than TP days as we would need time length periods more than TP days
    df_main = df_main[df_main['DAYS_SPENT'] >= TP]
    #Creating DAYS_TO_START feature to indicate days from START_DATE.
    df['DAYS_TO_START'] = pd.to_datetime(df['TRAN_DATE']) - pd.to_datetime(df['START_DATE'])
    # Convert days to numerical number.
    df['DAYS_TO_START'] = df['DAYS_TO_START'].astype('timedelta64[D]')
    #Drop all the othe DATE related columns as they will not be put into model
    df.drop(columns = ['TRAN_DATE', 'POST_DATE', 'OPEN_DATE', 'CLOSE_DATE', 'START_DATE'], inplace = True)
    #Save all the entries within time periods from the start of the matter TP
    df = df[df['DAYS_TO_START'] <= TP]
    #Creating PAST_ENGAGEMENT columns
    df['PAST_ENGAGEMENT'] = df['CLNT_MATT_CODE'].astype(str).str[-1].astype(int)
    df_past = df[['MATTER_UNO', 'PAST_ENGAGEMENT']].groupby('MATTER_UNO').mean()
    #Merge PAST_ENGAGEMENT columns to main table
    df_main = df_main.merge(df_past, on='MATTER_UNO', how = 'left')
    #Creating NOW_ENGAGEMENT columns within time periods
    df_now = pd.DataFrame(df[['MATTER_UNO','TIME_UNO']].groupby('MATTER_UNO').size()).rename(columns={0:'NOW_ENGAGEMENT'})
    #Merge NOW_ENGAGEMENT columns to main table
    df_main = df_main.merge(df_now, on='MATTER_UNO', how = 'left')
    #Creating PRE_TIMES columns within time periods
    df['PHASE_DESC'] = df['PHASE_DESC'].str.upper()
    df_PT = pd.DataFrame(df[df['PHASE_DESC'].str.contains('PRE-ACTION')].groupby('MATTER_UNO')
                         .size()).rename(columns = {0:'PRE_TIMES'})
    #Merge PRE_TIMES columns to main table
    df_main = df_main.merge(df_PT, on='MATTER_UNO', how = 'left')
    #Fill NULL values with mean in PRE_TIMES columns
    df_main['PRE_TIMES'].fillna(0, inplace=True)
    #Creating PRE_COST columns
    df_bill = df[['MATTER_UNO', 'TIME_UNO', 'BILLED_AMT']]
    #Sum up all the BILLED_AMT wihtin time periods
    df_bill = df_bill[['MATTER_UNO','BILLED_AMT']].groupby('MATTER_UNO').sum().rename(columns={'BILLED_AMT': 'PRE_COST'})
    #Merge the PRE_COST to main table
    df_main = df_main.merge(df_bill, on = 'MATTER_UNO', how = 'left')
    #Creating NUM_PPL
    #注意可能会有 collinarity 因为不同的position人数加起来就等于NUM_PPL
    df_num = pd.DataFrame(df[['MATTER_UNO','EMPL_UNO']].groupby('MATTER_UNO')['EMPL_UNO'].nunique())
    df_num.rename(columns={'EMPL_UNO' : 'NUM_PPL'}, inplace = True)

    #Merge to main table
    df_main = df_main.merge(df_num, on='MATTER_UNO', how ='left')
    #Merge MATT_DEPT_NAME to main table
    df_dept = df[['MATTER_UNO', 'MATT_DEPT_NAME']]
    #Drop dupliactes
    df_dept.drop_duplicates(subset=None, keep='first', inplace=True)
    #Merge into the main table
    df_main = df_main.merge(df_dept, on = 'MATTER_UNO', how = 'left')
    # One hot encoding MATT_DEPT_NAME column
    lb = LabelBinarizer()
    # Join the encoded df
    df_main = df_main.join(pd.DataFrame(lb.fit_transform(df_main['MATT_DEPT_NAME']),
                              columns=lb.classes_, 
                              index=df_main.index))
    # Drop one MATT_DEPT_NAME Column
    df_main.drop(columns = 'MATT_DEPT_NAME', inplace = True)
    # Creating features about all the different positions responsible for the matter within time periods. TP
    df_po = pd.DataFrame(df[['MATTER_UNO','EMPL_UNO']].groupby('MATTER_UNO')['EMPL_UNO'].unique()).reset_index()
    #Creating vector of different positions for each matter
    df_test = pd.DataFrame(df_po.EMPL_UNO.tolist(), index = df_po.MATTER_UNO)
    df_test.reset_index(inplace=True)
    df_test = df_test.fillna(0).applymap(int).applymap(str)
    # Convert unique EMPL_UNO of each matter to positons.
    df_test.replace({0: df_pos, 1: df_pos, 2: df_pos, 3: df_pos, 4: df_pos, 5: df_pos, 6: df_pos, 7: df_pos,
                     8: df_pos, 9: df_pos, 10: df_pos, 11: df_pos, 12: df_pos, 13: df_pos, 14: df_pos, 15: df_pos,
                     16: df_pos, 17: df_pos, 18: df_pos, 19: df_pos, 20: df_pos, 21: df_pos, 22: df_pos, 23: df_pos,
                     24: df_pos,  25: df_pos}
                   , inplace = True)
    # Creating position categories according to the meeting with supervisor
    Others = [ 'PSL', 'Costs',   'Compliance', 'Others (not billing)', 'Outdoor clerk', 'Consultant', 'Knowledge Services', 'EA & Projects Advisor', 'Secondee', 'Knowledge Analyst', 'eDisclosure', 'nan',  'Cyber', 'Senior Knowledge Analyst',   'Business Development Manager','Business Insight Manager',   'Senior Data Protection Specialist',  'Costs Lawyer',  'Consultant', 'Sports Lawyer',  'Knowledge Manager', 'Trainee Solicitor 2nd year', 'Trainee Solicitor (2nd yr)',  'Legal Executive', 'Knowledge Solutions Analyst', 'Project Manager', 'Legal Administrator',  'File Closing Support', 'Professional Support Lawyer', 'E-Disclosure Project Manager', 'Barrister', 'eDisclosure Project Manager',  'Cyber Inteligence Analyst',  'Cyber Intelligence Analyst', 'Cyber Innovation Architect',  'Cyber Manager',  'Trainee 2nd year', 'Cyber Developer', 'Junior Solutions Engineer', 'Intern', 'Temp', 'Senior Risk & Compliance Manager']
    Trainee = ['Trainee 2nd Year', 'Trainee 1st Year', 'Junior Solutions Engineer', 'Intern',  'Trainee Solicitor (2nd Year)',  'Trainee (Secondment)', 'Trainee']
    ParalegalPPA = ['PPA', 'Paralegal', 'Trade Mark Paralegal', 'Paralegal\\PA', 'Legal Assistant',  'Assistant Solicitor', 'Paralegal PA', 'Personal Assistant', 'Legal assistant', 'Paralegal/Personal Assistant',  'PA', 'Trade Mark Attorney',  'Litigation Recoveries Assistant']
    Associate = ['Indian Qualified Associate',  'Associate',  'In house compliance associate', 'Senior Associate',  'Associate (also BVI qualified)', 'Associate (Member of the New York Bar)', 'Associate (Barrister)', 'Associate, Chartered Trade Mark Attorney', 'Indian Qualified Associate']
    ManagingAssociate = ['Managing Associate, PSL', 'Managing Associate', 'Managing Associate (also BVI qualified)', 'Professional Support Lawyer - Managing Associate', 'Managing Associate, Professional Support Lawyer', 'Managing Associate (Russian qualified lawyer)', 'Managing Associate (Admitted at the Austrian Bar)']
    LegalDirector = ['Legal Director, PSL', 'Legal Director', 'Legal Director (Barrister)',   'Head of e Discovery',  'Head of Employment Policy & Engagement', 'Head of Legal Operations', 'China Desk Lead',  'Managing Associate (Russian qualified lawyer)',  'Professional Support Lawyer - Legal Director', 'Head of Legal Operations', 'Cyber Consulting Director', 'Cyber Intelligence Director', 'Head of Employment Policy & Engagement']
    Partner = ['Partner', 'Senior Equity Partner', 'Equity Partner', 'Partner (Junior)', 'General Counsel, Partner and COLP', 'Deputy Chairman', 'Senoir Equity Partner']
    # Replace all the positions with dictionary above
    df_test.replace(dict.fromkeys(Others, 'Others'), inplace = True)
    df_test.replace(dict.fromkeys(Trainee, 'Trainee'), inplace = True)
    df_test.replace(dict.fromkeys(ParalegalPPA, 'ParalegalPPA'), inplace = True)
    df_test.replace(dict.fromkeys(Associate, 'Associate'), inplace = True)
    df_test.replace(dict.fromkeys(ManagingAssociate, 'ManagingAssociate'), inplace = True)
    df_test.replace(dict.fromkeys(LegalDirector, 'LegalDirector'), inplace = True)
    df_test.replace(dict.fromkeys(Partner, 'Partner'), inplace = True)
    # Build up table to calculate all the positions in each matter
    df_pnum = df_test.apply(pd.Series.value_counts, axis=1)[['Others', 'Trainee', 'ParalegalPPA', 
                                                   'Associate','ManagingAssociate', 'LegalDirector', 'Partner']].fillna(0)
    # Join the table above to get the positions for each matter
    df_po_num = df_test.join(df_pnum)
    df_po_num = df_po_num[['MATTER_UNO', 'Others','Trainee','ParalegalPPA','Associate','ManagingAssociate','LegalDirector','Partner']]
    df_po_num = df_po_num.applymap(int)
    # Merge back to the main table
    df_main = df_main.merge(df_po_num, on = 'MATTER_UNO', how = 'left')
    # Drop all the matters from support department according to the communication with supervisor
    df_main.drop(df_main[df_main['Support']==1].index, inplace= True)
    # Classification
    df_main_cla = df_main.drop(['MATTER_UNO','TOTAL_AMT','DAYS_SPENT'], axis=1)
    # One Hot encoding the Value_RANGE
    encoder = LabelEncoder()
    cat = df_main_cla['VALUE_RANGE']
    cat_encoded = encoder.fit_transform(cat)
    encoder = OneHotEncoder()
    df_main_1hot = encoder.fit_transform(cat_encoded.reshape(-1,1))
    df_main_1hot = pd.DataFrame(df_main_1hot.toarray()).applymap(int)
    #Creat arrays for the features and the response variable
    y = df_main_cla['VALUE_RANGE'].values
    X = df_main_cla.drop('VALUE_RANGE', axis = 1).values
    # Create new training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    # Create the classifier: knn
    knn = KNeighborsClassifier(n_neighbors=7)
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(f"For {TP} days, KNN Model score is {score * 100:.2f}%")
    model = DummyClassifier(strategy='most_frequent')
    # fit model
    cv_results = cross_val_score(model, X, y, cv =5)
    print(cv_results)
    print("Accuracy of Model with Cross Validation is:",np.mean(cv_results) * 100)

In [5]:
# Testing 30, 60, 90 ,120, 150 days results of KNN model
for TP in range(30,151,30):
    accuracy(TP,df)

<ipython-input-4-38e50a865db0>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dept.drop_duplicates(subset=None, keep='first', inplace=True)


For 30 days, KNN Model score is 51.08%
[0.25179856 0.25270758 0.25270758 0.25270758 0.24909747]
Accuracy of Model with Cross Validation is: 25.180375555151546


<ipython-input-4-38e50a865db0>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dept.drop_duplicates(subset=None, keep='first', inplace=True)


For 60 days, KNN Model score is 61.23%
[0.25       0.25090909 0.25090909 0.25090909 0.24727273]
Accuracy of Model with Cross Validation is: 25.000000000000007


<ipython-input-4-38e50a865db0>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dept.drop_duplicates(subset=None, keep='first', inplace=True)


For 90 days, KNN Model score is 68.77%
[0.25650558 0.25746269 0.25746269 0.25746269 0.25746269]
Accuracy of Model with Cross Validation is: 25.7271264495367


<ipython-input-4-38e50a865db0>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dept.drop_duplicates(subset=None, keep='first', inplace=True)


For 120 days, KNN Model score is 79.92%
[0.26640927 0.26640927 0.26640927 0.26744186 0.26744186]
Accuracy of Model with Cross Validation is: 26.682230403160634


<ipython-input-4-38e50a865db0>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dept.drop_duplicates(subset=None, keep='first', inplace=True)


For 150 days, KNN Model score is 82.11%
[0.2804878  0.27642276 0.27642276 0.27642276 0.27642276]
Accuracy of Model with Cross Validation is: 27.723577235772357
